In [17]:
import pickle
from tqdm import tqdm
import pandas as pd

In [18]:
with open('data.pickle', 'rb') as f:
    data_dict = pickle.load(f)

In [19]:
print(data_dict)

icotemporal', 'RELATED'), ('zygomaticotemporal branch', 'RELATED'), ('zygomaticotemporal branch of zygomatic nerve', 'EXACT')], 'vulval vein': [], 'pelvic vein': [], 'penis blood vessel': [('penis vasculature', 'RELATED'), ('vasculature of penis', 'RELATED')], 'wall of left ventricle': [('left ventricular wall', 'EXACT'), ('pared del ventrC-culo izquierdo@es', 'EXACT')], 'wall of right ventricle': [('pared del ventrC-culo derecho@es', 'EXACT'), ('right ventricular wall', 'EXACT')], 'anterior wall of left ventricle': [], 'anterior wall of right ventricle': [], 'myocardium of anterior wall of left ventricle': [], 'myocardium of anterior wall of right ventricle': [], 'mucosa of lip': [('labial mucosa', 'EXACT')], 'renal pelvis/ureter': [('renal pelvis and ureter', 'EXACT'), ('renal pelvis plus ureter', 'EXACT')], 'vasculature of spleen': [], 'vasculature of central nervous system plus retina': [], 'vasculature of central nervous system': [], 'anatomical border': [], 'wall of coronary arte

In [20]:
count_list = []
for key, values in data_dict.items():
    for value in values:
        if ',' in value[0]:
            print(value[0])
        

T-lymphocyte
CD4-positive, alpha-beta memory T lymphocyte
CD4-positive, alpha-beta memory T-cell
CD4-positive, alpha-beta memory T-lymphocyte
naive thymus-dervied CD8-positive, alpha-beta T lymphocyte
naive thymus-dervied CD8-positive, alpha-beta T-cell
naive thymus-dervied CD8-positive, alpha-beta T-lymphocyte
central CD4-positive, alpha-beta memory T cell
central CD4-positive, alpha-beta memory T lymphocyte
central CD4-positive, alpha-beta memory T-cell
central CD4-positive, alpha-beta memory T-lymphocyte
central memory CD4-positive, alpha-beta T lymphocyte
central memory CD4-positive, alpha-beta T-cell
central memory CD4-positive, alpha-beta T-lymphocyte
effector CD4-positive, alpha-beta memory T cell
effector CD4-positive, alpha-beta memory T lymphocyte
effector CD4-positive, alpha-beta memory T-cell
effector CD4-positive, alpha-beta memory T-lymphocyte
effector memory CD4-positive, alpha-beta T lymphocyte
effector memory CD4-positive, alpha-beta T-cell
effector memory CD4-positive

In [21]:
import itertools
import random
# This function radomly sample negative pairs 
def generate_data(syno_sets):
    name1_list = []
    name2_list = []
    label_list = []
    for syno_set in syno_sets:
        for name1, name2 in itertools.product(syno_set, repeat=2):
            name1_list.append(name1)
            name2_list.append(name2)
            label_list.append(1)
        non_synonym_sets = random.sample([ x for x in syno_sets if x != syno_set], 3)
        non_synonym_list = [phrase for phrases in non_synonym_sets for phrase in phrases]
        for name1, name2 in itertools.product(syno_set, non_synonym_list):
            name1_list.append(name1)
            name2_list.append(name2)
            label_list.append(0)    
    return {'name1': name1_list, 'name2': name2_list, 'label': label_list}

In [22]:
import spacy
import string

# Declare special word/character.
START_W = '<w>'
END_W = '</w>'
START_S = '<s>'
END_S = '</s>'
DICT_SW = '</dsw>'
EOL_TK = '»'
UNK = '</unk>'
PAD = '</pad>'
RESERVE_TKS = [START_W, END_W, START_S, END_S, DICT_SW, UNK, PAD, EOL_TK]

PUCT_SET = set(string.punctuation)
STOP_WORDS = set(['an', 'the', 'and', 'of', 'in', 'on', 'at',
                  'other', 'others', 'The', 'nos', 'NOS'])
STOP_TOKENS = STOP_WORDS | PUCT_SET

spacy_model = spacy.load("en_core_web_sm")


def preprocess_name(name):
    parsed_result = spacy_model(name, disable=['parser', 'tagger', 'ner'])
    tokens = [str(_) for _ in parsed_result]
    tokens = [_ for _ in tokens if _.strip() != '' and _.lower() not in STOP_TOKENS]
    new_name = ' '.join(tokens)
    return name.lower() if new_name == '' else new_name.lower()

In [23]:
synonym_groups = []
filtered_group = []
synonym_groups_raw = []

for name, synonyms in tqdm(data_dict.items()):
    group = []
    group_raw = []

    group.append(preprocess_name(name))
    group_raw.append(name)
    for synonym in synonyms:
        if synonym[1] == 'EXACT':
            group.append(preprocess_name(synonym[0]))
            group_raw.append(synonym[0])
        else:
            filtered_group.append(synonym)
    synonym_groups.append(group)
    synonym_groups_raw.append(group_raw)

df_data = pd.DataFrame.from_dict(generate_data(synonym_groups))
df_data_exact = pd.DataFrame.from_dict(generate_data(synonym_groups_raw))

100%|██████████| 10032/10032 [00:00<00:00, 13661.35it/s]


In [24]:
df_data.head()

,name1,name2,label
0,continuant,continuant,1
1,continuant,intrahepatic bile duct,0
2,continuant,bile duct intrahepatic part,0
3,continuant,monocarboxylic acid metabolic process,0
4,continuant,monocarboxylate metabolic process,0


In [24]:
print(filtered_group[:10])

[('beta', 'RELATED'), ('beta(-)', 'RELATED'), ('beta-particle', 'RELATED'), ('e', 'RELATED'), ('e(-)', 'RELATED'), ('e-', 'RELATED'), ('Elektron', 'RELATED'), ('negatron', 'RELATED'), ('entidad molecular', 'RELATED'), ('entidades moleculares', 'RELATED')]


In [25]:
df_data.to_csv('pairwise_data_exact_only.csv', index = False)

In [28]:
df_data_exact.to_csv('pairwise_rawdata_exact_only', index = False)

In [26]:
print(len(df_data))
print(len(df_data_exact))
print(df_data[0:10])
print(df_data_exact[0:10])

410437
408569
        name1                                  name2  label
0  continuant                             continuant      1
1  continuant                 intrahepatic bile duct      0
2  continuant            bile duct intrahepatic part      0
3  continuant  monocarboxylic acid metabolic process      0
4  continuant      monocarboxylate metabolic process      0
5  continuant         monocarboxylic acid metabolism      0
6  continuant                           posterior_to      0
7   occurrent                              occurrent      1
8   occurrent                        oligodendrocyte      0
9   occurrent                                    ols      0
        name1                                          name2  label
0  continuant                                     continuant      1
1  continuant    negative regulation of leukocyte activation      0
2  continuant        down regulation of leukocyte activation      0
3  continuant        down-regulation of leukocyte acti

In [27]:
print(len(synonym_groups))
print(len(synonym_groups_exact))

10032
10032


ValueError: Sample larger than population or is negative

In [15]:
for i in tqdm(range(len(synonym_groups))):
    if len(synonym_groups[i]) != len(synonym_groups_exact[i]):
        print(synonym_groups[i], '\n')
        print(synonym_groups_exact[i], '\n')

100%|██████████| 10032/10032 [00:00<00:00, 1353097.01it/s]
